# Estimate diffusivity parameters from 1D b-space optimization
## Ryan Holmes

In [ ]:
# Load modules
import numpy as np
import h5py
import matplotlib
import shutil
import os
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
matplotlib.rcParams["figure.facecolor"] = "white"
matplotlib.rcParams["axes.facecolor"] = "white"
matplotlib.rcParams["savefig.facecolor"] = "white"
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import time
from IPython import display
import obsfit1d
from scipy.interpolate import interp1d

In [ ]:
# Do a single run for checks:
basedir = '/srv/ccrc/data03/z3500785/dedalus_Slope_Tracer/saveRUNS/prodruns24-5-19/';
run = basedir + 'z0_0p5000_AH_000_Kinf_m5_ADV_2_slope_400/'

# Get observed data:
params = np.load(run + 'runparams.npz');locals().update(params)
b = np.linspace(0,N2*np.cos(theta)*Lz + N2*np.sin(theta)*Ly,64);db = b[1]-b[0]
ba = (b[:-1]+b[1:])/2

with h5py.File(run + 'ifields.h5', mode='r') as file:
    B = file['tasks']['B'][0,:,:]
    B = (B[1:,:] + B[:-1,:])/2
    B = (B[:,1:] + B[:,:-1])/2
    y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];
dy = y[1:]-y[:-1];dz = z[1:]-z[:-1];
dA = np.tile(dy,[len(z)-1,1]).T*np.tile(dz,[len(y)-1,1]);

with h5py.File(run + 'snapshots.h5', mode='r') as file:

    t = file['scales/sim_time'][:]/lday;
    trT = file['tasks']['trT'][:,0,0]
    trF = np.zeros((len(b)-1,len(t)))
    for i in range(0,len(t)):#len(t),pfr):
        tr = file['tasks']['tr'][i,:,:]
        tr = (tr[1:,:] + tr[:-1,:])/2
        tr = (tr[:,1:] + tr[:,:-1])/2
        tr = tr*dA
        trF[:,i] = np.array([np.sum(tr[np.logical_and(B>=bi,B<(bi+db))])/db for bi in b[:-1]])

zF = ba/N2;
dzF = zF[1]-zF[0]
imax = np.argmax(trF[:,0])
zF = zF - zF[imax]

# Do 3-parameter fit:
#tinds = range(0,len(t))
tinds = range(0,20)
K = [];Kh = [];w = [];func = [];suc = [];
Kerr = []; Kherr = []; werr = [];msg = [];
for ti in tinds:
    if ((ti % 10) == 0):
        print('Doing time %03d of %03d, run %s' % (ti, len(t), run))
    res = obsfit1d.fit3par(zF,trF[:,ti],int(t[ti]*1e5),sz)
    obsfit1d.plot(res.x,zF,trF[:,ti],int(t[ti]*1e5),sz)
    K.append(res.x[0])
    Kh.append(res.x[1])
    w.append(res.x[2])
    Kerr.append(res.xerr[0])
    Kherr.append(res.xerr[1])
    werr.append(res.xerr[2])
    suc.append(res.success)
    func.append(res.fun)
    msg.append(res.msg)

K = np.array(K)
Kerr = np.array(Kerr)

f = plt.figure(figsize=(10,10),facecolor='white')
plt.subplot(2,1,1)
plt.plot(tinds,Kerr/K,'xk')
#cor = [x <= 0.00001 for x in func]
#incor = [x > 0.00001 for x in func]
plt.subplot(2,1,2)
tinds = np.array(tinds);
plt.plot(t[tinds],K,'xk');
plt.plot(t[tinds],K+2*Kerr,'xr')
plt.plot(t[tinds],K-2*Kerr,'xb')


In [ ]:
suc

In [ ]:
msg

In [ ]:
# Process multiple runs:
basedir = '/srv/ccrc/data03/z3500785/dedalus_Slope_Tracer/saveRUNS/prodruns24-5-19/';
folds = [basedir + x + '/' for x in os.listdir(basedir)];
for run in folds:

    # Get observed data:
    params = np.load(run + 'runparams.npz');locals().update(params)
    b = np.linspace(0,N2*np.cos(theta)*Lz + N2*np.sin(theta)*Ly,64);db = b[1]-b[0]
    ba = (b[:-1]+b[1:])/2

    with h5py.File(run + 'ifields.h5', mode='r') as file:
        B = file['tasks']['B'][0,:,:]
        B = (B[1:,:] + B[:-1,:])/2
        B = (B[:,1:] + B[:,:-1])/2
        y = file['scales/y/1.0'][:];z = file['scales/z/1.0'][:];
    dy = y[1:]-y[:-1];dz = z[1:]-z[:-1];
    dA = np.tile(dy,[len(z)-1,1]).T*np.tile(dz,[len(y)-1,1]);

    with h5py.File(run + 'snapshots.h5', mode='r') as file:

        t = file['scales/sim_time'][:]/lday;
        trT = file['tasks']['trT'][:,0,0]
        trF = np.zeros((len(b)-1,len(t)))
        for i in range(0,len(t)):#len(t),pfr):
            tr = file['tasks']['tr'][i,:,:]
            tr = (tr[1:,:] + tr[:-1,:])/2
            tr = (tr[:,1:] + tr[:,:-1])/2
            tr = tr*dA
            trF[:,i] = np.array([np.sum(tr[np.logical_and(B>=bi,B<(bi+db))])/db for bi in b[:-1]])

    zF = ba/N2;
    dzF = zF[1]-zF[0]
    imax = np.argmax(trF[:,0])
    zF = zF - zF[imax]

    # Do 3-parameter fit:
    tinds = range(0,len(t))
    K = [];Kh = [];w = [];func = [];suc = [];
    Kerr = []; Kherr = []; werr = [];
    for ti in tinds:
        if ((ti % 10) == 0):
            print('Doing time %03d of %03d, run %s' % (ti, len(t), run))
        res = obsfit1d.fit3par(zF,trF[:,ti],int(t[ti]*1e5),sz)
        # obsfit1d.plot(res.x,zF,trF[:,ti],int(t[ti]*1e5),sz)
        K.append(res.x[0])
        Kh.append(res.x[1])
        w.append(res.x[2])
        Kerr.append(res.xerr[0])
        Kherr.append(res.xerr[1])
        werr.append(res.xerr[2])
        suc.append(res.success)
        func.append(res.fun)

    # Save to file:
    np.savez(run + 'par3fit',K=K,Kh=Kh,w=w,func=func,suc=suc,Kerr=Kerr,Kherr=Kherr,werr=werr,t = t[tinds])